### Project parameters

In [16]:
## Imports ##
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras import layers, optimizers, Model
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.initializers import RandomUniform
from keras.utils.vis_utils import model_to_dot

from IPython.display import display, SVG

## Constants
num_classes = 10
img_rows, img_cols, img_chan = 28, 28, 1
img_shape = (img_rows, img_cols, img_chan)

metrics = ['accuracy']
verbose = 0


## Data ##
(x_reader, y_reader), (_, _) = mnist.load_data()
x_writer, y_writer = x_reader, y_reader

x_reader = x_reader.reshape(x_reader.shape[0], img_rows * img_cols)
x_reader = x_reader.astype(np.float32) / 255.0

x_writer = (x_writer.astype(np.float32) - 127.5) / 127.5 # Rescale -1 to 1
x_writer = np.expand_dims(x_writer, axis=3)

y_reader = to_categorical(y_reader, num_classes)
y_writer = y_writer.reshape(-1, 1)

history = [[],[],[],[]]


## Hyperparameters ##

# Reading
reader_epochs = 40
reader_validation_split = 0.2
reader_batch_size = 512
reader_kernel_initializer = RandomUniform(minval=0.0000001, 
                                          maxval=0.0001, 
                                          seed=None)
reader_optimizer = optimizers.Adam()

# Writing
writer_epochs = 22000
writer_latent_dim = 100
writer_batch_size = 32
writer_half_batch = int(batch_size / 2)
student_optimizer = optimizers.Adam(lr=0.0002,      # lr default:         0.001
                                    beta_1=0.5,     # beta_1 default:     0.9
                                    beta_2=0.999, 
                                    epsilon=None, 
                                    decay=0.0, 
                                    amsgrad=False)
grader_optimizer = student_optimizer
classroom_optimizer = grader_optimizer

print("Matrix Shapes:")
print("img_shape:\t", img_shape)
print('x_reader:\t', x_read.shape)
print('y_reader:\t', y_read.shape)
print('x_writer:\t', x_write.shape)
print('y_writer:\t', y_write.shape)

Matrix Shapes:
img_shape:	 (28, 28, 1)
x_reader:	 (60000, 784)
y_reader:	 (60000, 10)
x_writer:	 (60000, 28, 28, 1)
y_writer:	 (60000, 1)


# Models

### Reading model

In [1]:
reader = Sequential()
reader.add(Dense(320, activation='relu', 
                kernel_initializer=reader_kernel_initializer,
                input_shape=[x_reader.shape[1]]))
reader.add(Dropout(0.66))
reader.add(Dense(num_classes, activation='softmax'))
reader.compile(loss=categorical_crossentropy, 
               optimizer=reader_optimizer, 
               metrics=metrics)

NameError: name 'Sequential' is not defined

###

In [ ]:
reader_hist = reader.fit(x_train, y_train, 
                         epochs=reader_epochs, 
                         batch_size=reader_batch_size, 
                         verbose=verbose, 
                         validation_split=validation_split)